In [1]:
import pandas as pd
from sklearn.svm import LinearSVC
#df = pd.read_csv('Consumer_Complaints.csv')
df = pd.read_csv('14000_set.csv', encoding='latin1')

In [2]:
df = df[pd.notnull(df['Consumer complaint narrative'])]

In [3]:
col = ['Product', 'Consumer complaint narrative']
df = df[col]
df.columns = ['Product', 'Consumer_complaint_narrative']

In [4]:
df.columns = ['Product', 'Consumer_complaint_narrative']
ind_drop = df[df['Product'].apply(lambda x: x.startswith(('BMC issue' , 'Wallboard issue' , 'BMC issue' , 'Citrix issue' , 'Greenplum Issues' , 'IPCM Error' , 'IPCM error' , 'Missing Folder in MBNL' , 'Monolith panel error' , 'Network Protect Issues' , 'Other Issues' , 'Pegaplan error' , 'RAN request form' , 'Talend Issues' )))].index
df.drop(ind_drop, inplace=True)
df.loc[df['Product'].apply(lambda x: x.startswith(('Account Reset','Log in Issues' )))] = 'Account Locked'
df.loc[df['Product'].apply(lambda x: x.startswith(('Tool set access request','tool set access request' )))] = 'New Account'

In [5]:
df['category_id'] = df['Product'].factorize()[0]
df.head()

,Product,Consumer_complaint_narrative,category_id
0,Account Locked,"Hi, Kindly unlock the mentioned IP...",0
1,Account Locked,"Good Morning, Please see the update fro...",0
2,Account Locked,Hi Can you please assist with this? ...,0
3,Account Locked,Hi team I can??t login to SIAE NMS ...,0
4,Account Locked,"Hi Team, Can you please reset my U20...",0


In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emueana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from nltk.stem import WordNetLemmatizer
import re

stemmer = WordNetLemmatizer()

def cleanup(message):
    
    # Remove all the numbers
    document = re.sub("\d+", ' ', str(message))
    
    # Remove all the special characters
    document = re.sub('[^A-Za-z0-9]+', ' ', document)
    
    # remove all single characters
    for i in document.split():
        if len(i) == 1:
            document = document.replace(i , '')

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document


def remove_signature(message):
    string = " ".join(str(message).split())

    if 'Thank' in string:
        return(cleanup(string.split('Thank')[0]))
    elif 'thank' in string:
        return(cleanup(string.split('thank')[0]))
    elif 'Best Regard' in string:
        return(cleanup(string.split('Best Regard')[0]))
    elif 'Best regard' in string:
        return(cleanup(string.split('Best regard')[0]))
    elif 'Regard' in string:
        return(cleanup(string.split('Regard')[0]))
    elif 'regard' in string:
        return(cleanup(string.split('regard')[0]))
    elif 'BR' in string:
        return(cleanup(string.split('BR')[0]))
    else:
        return(cleanup(message))
    
df['New_Consumer_complaint_narrative'] = df['Consumer_complaint_narrative'].apply(remove_signature)

df.head()

#df[['Product', 'New Consumer complaint narrative','Consumer complaint narrative']].to_csv(r'C:\Users\ezmanhi\Desktop\EGI-Emaildata\test_email_complain.csv', index=False)

,Product,Consumer_complaint_narrative,category_id,New_Consumer_complaint_narrative
0,Account Locked,"Hi, Kindly unlock the mentioned IP...",0,hi kindly unlock the mentioned ipcm account us...
1,Account Locked,"Good Morning, Please see the update fro...",0,good morning plese see the updte from the supp...
2,Account Locked,Hi Can you please assist with this? ...,0,hi can you please assist with this am unable t...
3,Account Locked,Hi team I can??t login to SIAE NMS ...,0,hi eam can login o sae nm maser wed nm applica...
4,Account Locked,"Hi Team, Can you please reset my U20...",0,hi team can you please reset my password a am ...


In [8]:
from io import StringIO
import pickle
category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Product']].values)
print(id_to_category)
with open("category1.pkl", 'wb') as file:
    pickle.dump(id_to_category, file)

{0: 'Account Locked', 1: 'New Account', 2: 'Password Reset', 3: 'New account'}


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2),stop_words='english')
features = tfidf.fit_transform(df.New_Consumer_complaint_narrative).toarray()

In [10]:
labels = df.category_id
features.shape

(3407, 1069)

In [15]:
from sklearn.feature_selection import chi2
import numpy as np

N = 3
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:])))

# 'Account Locked':
  . Most correlated unigrams:
       . ount
       . new
       . locked
  . Most correlated bigrams:
       . prout deult
       . new account
       . account locked
# 'New Account':
  . Most correlated unigrams:
       . account
       . locked
       . new
  . Most correlated bigrams:
       . reset password
       . account locked
       . new account
# 'New account':
  . Most correlated unigrams:
       . requesting
       . create
       . dear
  . Most correlated bigrams:
       . account created
       . create new
       . dear team
# 'Password Reset':
  . Most correlated unigrams:
       . password
       . new
       . account
  . Most correlated bigrams:
       . reset password
       . account locked
       . new account


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['New_Consumer_complaint_narrative'], df['Product'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
model = LinearSVC()
model = CalibratedClassifierCV(LinearSVC()) 

In [12]:
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train,)
y_pred = model.predict_proba(X_test)
print(y_pred)

[[3.55254485e-02 1.51428156e-02 9.47333267e-01 1.99846878e-03]
 [1.27907329e-01 8.32886814e-01 3.78359119e-02 1.36994441e-03]
 [1.76626446e-02 9.74994143e-01 5.37386792e-03 1.96934437e-03]
 ...
 [8.71517694e-02 1.09974202e-01 8.01051652e-01 1.82237622e-03]
 [1.76626446e-02 9.74994143e-01 5.37386792e-03 1.96934437e-03]
 [2.34217016e-02 7.17542970e-03 9.69092446e-01 3.10422792e-04]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [13]:
model.fit(features, labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid')

In [16]:
# serialize weights to HDF5
import pickle
pickle.dump(model, open("model1.pkl", 'wb'))
print("Saved model to disk")

Saved model to disk


In [17]:
pickle.dump(tfidf, open("tfidf1.pkl", 'wb'))
print("Saved tfidf model to disk")

Saved tfidf model to disk


In [18]:
# load the model from disk
loaded_model = pickle.load(open("model1.pkl", 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9688888888888889


In [19]:
# load tfidf model from disk
loaded_tfidf = pickle.load(open("tfidf1.pkl", 'rb'))

In [20]:
text = ["Could you please renew my MBNL Citrix Password?"]
text_features = loaded_tfidf.transform(text)
predictions = loaded_model.predict_proba(text_features)

In [21]:
import json
indices = np.where(predictions == predictions.max())
for predicted in predictions:
    with open("category1.pkl", 'rb') as file:
        category = pickle.loads(file.read())
        print(category)
    print("Predicted as: %s"%category[indices[1][0]])
    print("Confidence : %s%%"%(predictions.max()*100))

{0: 'Account Locked', 1: 'New Account', 2: 'Password Reset', 3: 'New account'}
Predicted as: Password Reset
Confidence : 92.22556819295205%


In [22]:
result =['Node   Auth Algo   Auth Password   Priv Algo   Priv Password' , 'MUN-ECAA042   SHA1    enmuserauthkey  DES enmuserprivkey']
ret = ' '.join(result[1].split()).split(' ')[2]
print (ret)

enmuserauthkey
